In [1]:
!pip install pandas

In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.41.3  transformers==4.31.0 trl==0.4.7

In [3]:
!pip install torch
!pip install scipy
!pip install transformers accelerate
!pip install --upgrade accelerate
!pip install tensorboardx
!pip install ipywidgets
!pip install huggingface_hub

  Using cached accelerate-1.0.1-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.0.1-py3-none-any.whl (330 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [4]:
import numpy as np

In [5]:
import pandas as pd

In [6]:
model_df=pd.read_csv('model_df_balanced.csv')
model_df

,TEXT,readmitted
0,neonatology attending triage notebaby name ni ...,0
1,2101106 602 pm chest portable ap clip clip num...,0
2,2191315 420 pm chest portable ap clip clip num...,0
3,nnp triage notebb known lastname 6 delivered 4...,0
4,2175525 1052 chest preop pa lat clip clip numb...,0
...,...,...
81303,baseline artifactprobable atrial fibrillation ...,1
81304,sinus bradycardia probable left atrial abnorma...,1
81305,2122320 221 pm chest preop pa lat clip clip nu...,1
81306,21031029 1044 pm babygram chest clip clip numb...,1


In [7]:
sampled_df = model_df.sample(n=1500, random_state=42)

In [8]:
sampled_df

,TEXT,readmitted
51320,2127217 132 pm chest preop pa lat clip clip nu...,1
80965,nursing admit note1 infant potential sepsis2 a...,1
4569,2183118 826 chest portable ap clip clip number...,0
11181,sinus rhythmnormal ecg 2112616 643 pm chest po...,0
48414,sinus rhythm ventricular bigeminy previous tra...,1
...,...,...
7615,217732 522 pm cta head wwo c recons ct 150cc n...,0
19730,sinus bradycardiaindeterminate frontal qrs axi...,0
68932,neonatologyknown lastname 2188 known lastname ...,1
38254,216488 300 pm chest preop pa lat clip clip num...,0


In [9]:
# Additional package installations
!pip install transformers bitsandbytes
!pip install colored
!pip install --upgrade transformers

  Using cached transformers-4.46.1-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.46.1-py3-none-any.whl (10.0 MB)
Using cached tokenizers-0.20.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [10]:
# Load the model with 4-bit quantization
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Meta-Llama-3.1-8B-Instruct",
    quantization_config=quantization_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3.1-8B-Instruct")


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [14]:
# If the tokenizer does not have a pad token, set it to eos_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

# Define the tokenization function for the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

# Tokenize the dataset
my_dataset = my_dataset.map(tokenize_function, batched=True, remove_columns=my_dataset.column_names)
my_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [11]:
# Define the question template and data generation
import random
from datasets import Dataset

TEMPLATE = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{context}<|eot_id|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{answer}<|eot_id|>"""

QUESTION_TEMPLATES = [
    "Given the patient's symptoms of {symptoms}, is further hospitalization required?",
    "Considering the patient's history of {condition}, will they need to be readmitted?",
    "With the treatment of {treatment}, will the patient return to the hospital?",
    "Does the presence of {issue} indicate that the patient will be readmitted?",
]

def generate_question(context):
    symptoms = random.choice(["fever", "chest pain", "shortness of breath", "dizziness"])
    condition = random.choice(["diabetes", "hypertension", "heart failure", "kidney issues"])
    treatment = random.choice(["medication", "surgery", "rehabilitation", "dialysis"])
    issue = random.choice(["complications", "poor recovery", "infection", "pain"])
    question = random.choice(QUESTION_TEMPLATES).format(symptoms=symptoms, condition=condition, treatment=treatment, issue=issue)
    return question

data = []

for index, row in sampled_df.iterrows():
    context = row['TEXT']
    question = generate_question(context)
    answer = "Yes, the patient will be readmitted." if row['readmitted'] == 1 else "No, the patient will not be readmitted."
    data.append({"text": TEMPLATE.format(context=context, question=question, answer=answer)})

my_dataset = Dataset.from_list(data)


In [12]:
# Login to Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()

In [18]:
pip install --upgrade transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.46.1-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.46.1-py3-none-any.whl (10.0 MB)
Using cached tokenizers-0.20.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.1
    Uninstalling transformers-4.28.1:
      Successfully uninstalled transformers-4.28.1
Note: you may need to restart the kernel to use updated packages.


In [20]:
!pip uninstall -y transformers trl


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: transformers 4.46.1
Uninstalling transformers-4.46.1:
  Successfully uninstalled transformers-4.46.1
Found existing installation: trl 0.4.7
Uninstalling trl-0.4.7:
  Successfully uninstalled trl-0.4.7


In [21]:
!pip install transformers trl


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.46.1-py3-none-any.whl.metadata (44 kB)
  Using cached trl-0.11.4-py3-none-any.whl.metadata (12 kB)
Using cached transformers-4.46.1-py3-none-any.whl (10.0 MB)
Using cached trl-0.11.4-py3-none-any.whl (316 kB)


In [22]:
import transformers
import trl

print("Transformers version:", transformers.__version__)
print("TRL version:", trl.__version__)


Transformers version: 4.46.1
TRL version: 0.11.4


In [23]:
# Configure LoRA and training arguments
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="checkpoints",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=100,
    learning_rate=2e-4,
    logging_steps=10,
    optim="adamw_torch",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=1234,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=my_dataset,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=256,
    packing=False,
)

# Start training
trainer.train()

# Save the final model
final_model_name = "LLAMA3.1_Hospital_RAG"
trainer.save_model(final_model_name)
print(f"Final model saved to: {final_model_name}")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. 

Step,Training Loss
10,4.537600
20,3.976800
30,3.748800
40,3.603800
50,3.403900
60,3.313100
70,3.270600
80,3.210000
90,3.212100
100,3.145900


Final model saved to: LLAMA3.1_Hospital_RAG
